In [1]:
import numpy as np
import pandas as pd
import time
import cdsw
import cmlapi
import os
import json

In [2]:
if os.environ.get("MODEL_NAME") == "":
    os.environ["MODEL_NAME"] = "lstm"
if os.environ.get("PROJECT_NAME") == "":
    os.environ["PROJECT_NAME"] = "SDWAN"

In [3]:
# model_name = os.environ.get("MODEL_NAME")
# project_name = os.environ.get("PROJECT_NAME")

#model_name = "lstm"
model_name = "lstm-test"
project_name = "SDWAN"

client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
target_model = client.list_all_models(search_filter=json.dumps({"name": model_name}))

proj_id = client.list_projects(search_filter=json.dumps({"name":project_name })).projects[0].id

mod_id = target_model.models[0].id

build_list = client.list_model_builds(project_id = proj_id, model_id = mod_id,sort='created_at')

build_id = build_list.model_builds[-1].id
model_deployments = client.list_model_deployments(project_id=proj_id,model_id=mod_id,build_id=build_id)
cr_number = model_deployments.model_deployments[0].crn

In [7]:
# write a function to print variable crn
metrics = cdsw.read_metrics(model_deployment_crn=cr_number,dev=False)

In [23]:
metrics['metrics'][1]

{'modelDeploymentCrn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:55da5a39-10bf-48bc-8d2c-6436308fdff9/282b596f-0f63-46e3-97df-45c9886d071a',
 'modelBuildCrn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:55da5a39-10bf-48bc-8d2c-6436308fdff9/013ee7ed-697c-4c47-bd09-df4da963b82a',
 'modelCrn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:55da5a39-10bf-48bc-8d2c-6436308fdff9/b5ef7e5d-3ef2-464d-b9b7-245383ece40e',
 'startTimeStampMs': 1715887221756,
 'endTimeStampMs': 1715887222489,
 'predictionUuid': '22bc560a-a5a7-44b2-bdd7-0679aa28fec7',
 'metrics': {'final_rx_gbs_label': [9.315300445,
   1.493328951,
   1.798052112,
   0.862901636,
   2.699306281,
   9.293342977],
  'final_tx_gbs_label': [2.575881772,
   10.520229461,
   2.379724223,
   9.006671892,
   11.488075298,
   10.179279896],
  'forecast time': ['2024-05-09T01:00:00',
   '2024-05-09T02:00:00',
   '2024-05-09T03:00:00',
   '2024-05-09T04:00:00',
   '2024-

In [8]:
performance_metrics = []
core_metrics = []
for item in metrics['metrics']:
    if len(item['metrics'].keys()) < 5:
         performance_metrics.append(item['metrics'])
    else:
        core_metrics.append(item['metrics'])

In [9]:
len(core_metrics)

59

In [10]:
#core_metrics[0]

In [11]:
metrics_df = pd.json_normalize(core_metrics)

In [16]:
metrics_df.tail()

,final_rx_gbs_label,final_tx_gbs_label,forecast time,input_rx_data,input_tx_data,rx bytes prediction,tx bytes prediction
54,"[1.411229685, 1.305595741, 1.413211706, 10.531...","[10.606051833, 5.167073481, 2.53719506, 9.0066...","[2024-05-13T01:00:00, 2024-05-13T02:00:00, 202...","[1.273866347, 1.342958131, 8.140159838, 1.9908...","[4.014604812, 1.955984192, 2.886702137, 2.7588...","[0.8348015546798706, 0.8849822282791138, 0.918...","[0.057432450354099274, 0.0744861513376236, 0.0..."
55,"[1.094988562, 0.889843661, 3.747575266, 1.7418...","[1.43889315, 4.661999297, 11.812364196, 5.2579...","[2024-05-02T01:00:00, 2024-05-02T02:00:00, 202...","[2.465220145, 2.280574848, 1.70296266, 1.56591...","[9.689268193, 11.636575472, 3.688097012, 10.63...","[0.6253072023391724, 0.688372015953064, 0.7331...","[0.08840098977088928, 0.0815410241484642, 0.07..."
56,"[4.123924896, 9.623044791, 6.396868241, 1.7796...","[3.808545758, 2.429849325, 4.201446881, 3.0185...","[2024-05-17T01:00:00, 2024-05-17T02:00:00, 202...","[1.646113664, 0.567956557, 4.123836151, 2.1252...","[4.163275395, 2.391207379, 10.021471138, 5.015...","[0.5600473880767822, 0.614625096321106, 0.6599...","[0.06582796573638916, 0.06737610697746277, 0.0..."
57,"[9.711008288, 0.6623837619999999, 1.05802475, ...","[3.386650789, 7.865282364, 1.655461957, 5.5431...","[2024-05-08T01:00:00, 2024-05-08T02:00:00, 202...","[1.017495761, 6.484177698, 0.571594764, 8.3529...","[11.23759017, 1.6609205299999998, 11.11969373,...","[0.5358205437660217, 0.6750768423080444, 0.809...","[0.12858328223228455, 0.15729646384716034, 0.1..."
58,"[0.567956557, 4.123836151, 2.125209277, 1.0186...","[2.391207379, 10.021471138, 5.015414073, 3.850...","[2024-05-16T01:00:00, 2024-05-16T02:00:00, 202...","[1.290738923, 3.540820261, 0.855667453, 3.8402...","[2.422788214, 4.430113012, 4.592582602, 13.240...","[0.4909769892692566, 0.6044356226921082, 0.665...","[0.0815158262848854, 0.09619943797588348, 0.11..."


In [15]:
metrics_df.isnull().sum()

final_rx_gbs_label     1
final_tx_gbs_label     1
forecast time          0
input_rx_data          0
input_tx_data          0
rx bytes prediction    0
tx bytes prediction    0
dtype: int64

In [19]:
metrics_df[metrics_df.isna().any(axis=1)]

,final_rx_gbs_label,final_tx_gbs_label,forecast time,input_rx_data,input_tx_data,rx bytes prediction,tx bytes prediction
27,NaN,NaN,"[2024-05-31T01:00:00, 2024-05-31T02:00:00, 202...","[9.142250296, 11.37641808, 1.391518371, 1.6188...","[13.741614516, 1.444370114, 4.774979732, 11.43...","[0.4038369059562683, 0.4840385317802429, 0.551...","[0.10492565482854843, 0.09039974957704544, 0.1..."
